In [27]:
from io import open
import pickle
import sys
sys.path.insert(0,'./zipped_files_from_www')
sys.path.insert(0,'./Boyu')
sys.path.insert(0,'./searchCatDistData')
from posterior import *
from sklearn.metrics import average_precision_score,precision_recall_fscore_support,confusion_matrix, roc_curve, auc, classification_report, average_precision_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from psi_npsi_user_mapping import *
from helper import *
import torch
#from random_slice import *
cats = ["Business & Industrial","Home & Garden","Travel","Arts & Entertainment","Sports","Food & Drink","Pets & Animals","Health","Shopping","Finance","Adult","Beauty & Fitness","News","Books & Literature","Online Communities","Law & Government","Sensitive Subjects","Science","Hobbies & Leisure","Games","Jobs & Education","Autos & Vehicles","Computers & Electronics","People & Society","Reference","Internet & Telecom","Real Estate"]

In [6]:
ls_compound, nls_compound = load_pickle("Boyu/compound_vectors_self_esteem.pkl")
psi_compound, npsi_compound = load_pickle('Boyu/compound_vectors_psi.pkl')
#ls_users = [i[0] for i in ls_cat]
#len(ls_cat),len(ls_lamb),len(nls_cat),len(nls_lamb)

In [9]:
psi_users = [i[0] for i in psi_compound]
npsi_users = [i[0] for i in npsi_compound]
ls_users = [i[0] for i in ls_compound]

In [3]:
ls, nls = load_pickle('Boyu/best_representation.pkl')

In [30]:
def load_data_MLP_with_userid(path, scale):

    with open(path, 'rb') as f:
        (low_list, not_low_list) = pickle.load(f)
    # [number of users, number of features]
    ls = np.stack([user[1] * scale for user in low_list])
    nls = np.stack([user[1] * scale for user in not_low_list])

    ls_users = np.stack([[user[0]] for user in low_list])
    nls_users = np.stack([[user[0]] for user in not_low_list])
    print('low shape: [{}], not low shape: [{}]'.format(ls.shape, nls.shape))
    assert ls.shape[1] == nls.shape[1]
    in_size = ls.shape[1]
    ls_num = ls.shape[0]
    nls_num = nls.shape[0]

    # add label [number of users, number of features]
    ls_label = np.stack([np.asarray([1, 0]) for user in ls])
    ls = np.concatenate((ls_users,ls, ls_label), axis = 1)
    nls_label = np.stack([np.asarray([0, 1]) for user in nls])
    nls = np.concatenate((nls_users,nls, nls_label), axis = 1)
    # print('ls example: [{}], nls example: [{}]'.format(ls[0], nls[0]))

    # [number of users, number of features + number of labels]
    data = np.concatenate((ls, nls), axis = 0)
    np.random.shuffle(data)
    #data = torch.from_numpy(data).float()
    return data, in_size, ls_num, nls_num


In [33]:
data, in_size, ls_num, nls_num= load_data_MLP_with_userid(
		path = 'Boyu/lambda_vectors_with_user_ID.pkl', 
		scale = 10**5)



low shape: [(49, 27)], not low shape: [(43, 27)]


In [34]:
data

array([['70_361042a326fdc0', '0.3427522120783459', '0.1328910684930576',
        ..., '0.03067144607031011', '1', '0'],
       ['22_35fd5d66c7e2f0', '0.009197232188624047',
        '0.004741988185715794', ..., '0.0', '1', '0'],
       ['65_36102f386840b2', '0.24810390543574717',
        '0.08498117681192202', ..., '0.05166916319905795', '1', '0'],
       ...,
       ['75_3611064f3a7244', '0.9011333253455095', '0.30932692417884644',
        ..., '0.15271480077022642', '1', '0'],
       ['1_35f88eac597396', '0.18957064952461003',
        '0.054462875724297595', ..., '0.018580721398878956', '0', '1'],
       ['16_35fb0466ef53b6', '0.0', '0.0', ..., '0.0', '1', '0']],
      dtype='<U32')